In [ ]:
from matplotlib.colors import SymLogNorm
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from astropy.coordinates.sky_coordinate import SkyCoord
import astropy.units as u
from astropy.cosmology import Planck15 as cosmo
from astropy.wcs import WCS
from minot import model

# Modify plotting parameters
dict_base = {'font.size':        16,
             'legend.fontsize':  16,
             'xtick.labelsize':  16,
             'ytick.labelsize':  16,
             'axes.labelsize':   16,
             'axes.titlesize':   16,
             'figure.titlesize': 16,    
             'figure.figsize':[8.0, 6.0],
             'figure.subplot.right':0.97,
             'figure.subplot.left':0.15,
             'font.family':'serif',
             'figure.facecolor': 'white',
             'legend.frameon': True}
plt.rcParams.update(dict_base)

In [ ]:
clust = model.Cluster(name='A3558', 
                      redshift=0.048, M500=6.71e14*u.Msun, 
                      cosmology=cosmo, silent=False, 
                      output_dir='/Users/vdk/Desktop/test')

In [ ]:
#----- Some parameters that can be set at when defining the cluster
clust.coord = SkyCoord("203.098d  -33.139d", frame="icrs", unit="deg")

#----- Define the relevant ICM model (available models are defined in model_modpar.py)

# Density profile of the target thermal gas
clust.density_gas_model = {'name':'beta', 'n_0':0.46e-2*u.cm**-3, 'beta':0.58, 'r_c':223*u.kpc}

# Pressure profile of the target thermal gas
#clust.pressure_gas_model = {'name':'GNFW', 'P_0':2.2e-2*u.keV/u.cm**3, 'c500':3.2, 'a':1.5, 'b':3.1, 'c':0.0}

#Exact from Planck 13
clust.pressure_gas_model = {'name':'GNFW', 'P_0':(6.04*0.0030143804978998274)*u.keV/u.cm**3, 'c500':1.77, 'a':1.12, 'b':3.58, 'c':0.31}

#Gong 20
#clust.pressure_gas_model = {'name':'GNFW', 'P_0':2.99e-2*u.keV/u.cm**3, 'c500':1.16, 'a':2.66, 'b':5.48, 'c':0.3}

#Plank Coma Universal test
#clust.pressure_gas_model = {'name':'GNFW', 'P_0':2.57e-2*u.keV/u.cm**3, 'c500':1.17, 'a':1.05, 'b':5.49, 'c':0.3}

# Set the cosmic ray density profile to the same shape as the thermal gas density
clust.set_density_crp_isodens_scal_param()

# Cosmic ray proton spectrum
clust.spectrum_crp_model = {'name':'PowerLaw', 'Index':2.2}

# Cosmic ray proton normalization
clust.X_crp_E = {'X':0.05, 'R_norm':clust.R500} # X is the CRp/thermal energy ratio, normalized within R_norm

#Set magnetic field profile
clust.set_magfield_isodens_scal_param(5*u.uG, scal=0.5)

In [ ]:
clust.print_param() # To print the parameters in the terminal
clust.save_param()  # To write the parameters in the output directory

In [ ]:
plt.figure(figsize=(15,6))
plt.subplot(121)
r, n_gas_r = clust.get_density_gas_profile()
plt.loglog(r, n_gas_r, label='Gas density')
plt.xlabel('radius (kpc)')
plt.ylabel('Density (cm$^{-3}$)')
plt.legend()

plt.subplot(122)
r, n_crp_r = clust.get_density_crp_profile()
plt.loglog(r, n_crp_r, label='CRp density (all energies)')
r, n_crp_r = clust.get_density_crp_profile(Emin=100*u.GeV, Emax=100*u.PeV)
plt.loglog(r, n_crp_r, '--', label='CRp density (100 GeV < E < 100 PeV)')
plt.xlabel('radius (kpc)')
plt.ylabel('Density (cm$^{-3}$)')
plt.legend()

In [ ]:
plt.figure(figsize=(10,8))
r, x_r = clust.get_crp_to_thermal_energy_profile(Emin=None, Emax=None)
plt.plot(r, x_r, label='Cosmic ray to thermal energy')
plt.vlines(clust.X_crp_E['R_norm'].to_value('kpc'), clust.X_crp_E['X']*0.9, clust.X_crp_E['X']*1.1, label='Imposed normalization')
plt.hlines(clust.X_crp_E['X'], clust.X_crp_E['R_norm'].to_value('kpc')*0.9, clust.X_crp_E['R_norm'].to_value('kpc')*1.1)
plt.xscale('log')
plt.xlabel('radius (kpc)')
plt.ylabel('$U_{CRp}/U_{th} (<r)$')
plt.legend()

In [ ]:
r1, p1 = clust.get_gamma_profile(radius=np.logspace(0,4,100)*u.kpc, 
                                   Emin=500*u.GeV, Emax=1*u.PeV, Energy_density=False)
r2, p2 = clust.get_gamma_profile(radius=np.logspace(0,4,100)*u.kpc, 
                                   Emin=1000*u.GeV, Emax=1*u.PeV, Energy_density=False)
plt.figure(0,figsize=(10,8))
plt.loglog((r1/clust.D_ang*u.rad).to('deg'),p1.to('cm-2 s-1 sr-1'), '-', label='500 GeV - 1 PeV')
plt.loglog((r2/clust.D_ang*u.rad).to('deg'),p2.to('cm-2 s-1 sr-1'), '--', label='1 TeV - 1 PeV')
plt.xlabel('Angular size (deg)')
plt.ylabel('Surface brightness ($ph/cm^2/s/sr$)')
plt.legend()

In [ ]:
energy = np.logspace(-1,5,100)*u.GeV
e1, s1 = clust.get_gamma_spectrum(energy,Rmax=clust.R_truncation, type_integral='spherical')
e2, s2 = clust.get_gamma_spectrum(energy,Rmax=clust.R500, type_integral='spherical')
e3, s3 = clust.get_gamma_spectrum(energy,Rmax=clust.R500, type_integral='cylindrical')

plt.figure(0,figsize=(10,8))

bins_lo = [0.02, 0.0316, 0.0501, 0.0794, 0.126, 0.2, 0.316, 0.501, 0.794, 1.26, 2, 3.16, 5.01, 7.94, 12.6, 20, 31.6, 50.1, 79.4, 126] # E_min (TeV)
bins_hi = [0.0316, 0.0501, 0.0794, 0.126, 0.2, 0.316, 0.501, 0.794, 1.26, 2, 3.16, 5.01, 7.94, 12.6, 20, 31.6, 50.1, 79.4, 126, 200] # E_max (TeV)
data = [2.4e-11, 7.65e-12, 2.16e-12, 1.04e-12, 4.63e-13, 2.98e-13, 1.86e-13, 1.26e-13, 9.76e-14, 7.83e-14, 6.39e-14,
        5.72e-14, 5.75e-14, 7.27e-14, 9.07e-14, 1.14e-13, 1.58e-13, 2.39e-13, 3.64e-13, 5.67e-13] # S (erg / cm^2 / s)
data_GeV = [i * 624.15064799632 for i in data]
mid_point = [np.exp((np.log(i)+np.log(j))/2) * 1000 for i, j in zip(bins_lo, bins_hi)]
plt.loglog(mid_point, data_GeV, color='k', ls='-.', lw='2', alpha=0.5, label = 'CTA South sensivity 50h')

plt.gca().set_ylim(bottom= 1e-12)
plt.gca().set_xlim(left= 1e-1)

plt.loglog(e1, (e1**2*s1).to('GeV cm-2 s-1'), '-', label='Total volume')
#plt.loglog(e2, (e2**2*s2).to('GeV cm-2 s-1'), '-', label='<R500, spherical integration')
#plt.loglog(e3, (e3**2*s3).to('GeV cm-2 s-1'), '-', label='<R500, cylindrical integration')
plt.xlabel('Energy (GeV)')
plt.ylabel('Flux ($GeV/cm^2/s$)')
plt.legend()

In [ ]:
# Defines the grid
clust.map_reso = 0.01*u.deg
clust.map_fov = 5.0*u.deg

# The center of the map can be offset wrt the cluster
clust.map_coord = SkyCoord("203.098d  -33.139d", frame="icrs", unit="deg")

# Get the corresponding header
header = clust.get_map_header()

In [ ]:
template = clust.get_gamma_map(Emin=None, Emax=None, 
                               Energy_density=False, 
                               Normalize=True) # The map is normalized to the total flux, to be in unit of sr-1

In [ ]:
test = SkyCoord('16h05m22.4s', '+16d26m55s', frame='icrs')

In [ ]:
test.icrs

In [ ]:
rad500 = clust.theta500.to_value('deg')
radtru = clust.theta_truncation.to_value('deg')

fig = plt.figure(figsize=(10, 8))
ax = plt.subplot(111, projection=WCS(header))
plt.imshow(template, vmin=0, origin='lower', cmap='magma', norm=SymLogNorm(1))

# Show R500
circle500 = matplotlib.patches.Ellipse((clust.coord.ra.value, clust.coord.dec.value),
                                        2*rad500/np.cos(clust.coord.dec.value*np.pi/180), 2*rad500,
                                        linewidth=2, fill=False, zorder=2,
                                        edgecolor='green', linestyle='dashed',facecolor='none',
                                        transform=ax.get_transform('fk5'))
# Show R_truncation
circletru = matplotlib.patches.Ellipse((clust.coord.ra.value, clust.coord.dec.value),
                                        2*radtru/np.cos(clust.coord.dec.value*np.pi/180), 2*radtru,
                                        linewidth=1, fill=False, zorder=2,
                                        edgecolor='green', linestyle='dashed',facecolor='none',
                                        transform=ax.get_transform('fk5'))
ax.add_patch(circle500)
ax.add_patch(circletru)
plt.colorbar()
plt.title(clust.name+', gamma template ($sr^{-1}$)')

In [ ]:
# Save the parameters
clust.save_param()

# Save relevant output products
clust.save_spectra()
clust.save_profile()
clust.save_map()

# Can also produce plots
clust.plot(prod_list=['all'])

In [ ]:
clust_two = model.Cluster(name='A3560', 
                      redshift=0.0495, M500=2.77e14*u.Msun, 
                      cosmology=cosmo, silent=False, 
                      output_dir='/Users/vdk/Desktop/test')

In [ ]:
#----- Some parameters that can be set at when defining the cluster
clust_two.coord = SkyCoord("203.098d  -33.139d", frame="icrs", unit="deg")

#----- Define the relevant ICM model (available models are defined in model_modpar.py)

# Density profile of the target thermal gas
clust_two.density_gas_model = {'name':'beta', 'n_0':0.46e-2*u.cm**-3, 'beta':0.58, 'r_c':223*u.kpc}

# Pressure profile of the target thermal gas
#clust_two.pressure_gas_model = {'name':'GNFW', 'P_0':2.2e-2*u.keV/u.cm**3, 'c500':3.2, 'a':1.5, 'b':3.1, 'c':0.0}

#Plank Coma Universal test
clust_two.pressure_gas_model = {'name':'GNFW', 'P_0':2.57e-2*u.keV/u.cm**3, 'c500':1.17, 'a':1.05, 'b':5.49, 'c':0.3}

#Gong 20
#clust_two.pressure_gas_model = {'name':'GNFW', 'P_0':2.99e-2*u.keV/u.cm**3, 'c500':1.16, 'a':2.66, 'b':5.48, 'c':0.3}

# Set the cosmic ray density profile to the same shape as the thermal gas density
clust_two.set_density_crp_isodens_scal_param()

# Cosmic ray proton spectrum
clust_two.spectrum_crp_model = {'name':'PowerLaw', 'Index':2.2}

# Cosmic ray proton normalization
clust_two.X_crp_E = {'X':0.05, 'R_norm':clust.R500} # X is the CRp/thermal energy ratio, normalized within R_norm

#Set magnetic field profile
clust_two.set_magfield_isodens_scal_param(5*u.uG, scal=0.5)

In [ ]:
energy = np.logspace(-1,5,100)*u.GeV
e1, s1 = clust_two.get_gamma_spectrum(energy,Rmax=clust.R_truncation, type_integral='spherical')
e2, s2 = clust_two.get_gamma_spectrum(energy,Rmax=clust.R500, type_integral='spherical')
e3, s3 = clust_two.get_gamma_spectrum(energy,Rmax=clust.R500, type_integral='cylindrical')

plt.figure(0,figsize=(10,8))

bins_lo = [0.02, 0.0316, 0.0501, 0.0794, 0.126, 0.2, 0.316, 0.501, 0.794, 1.26, 2, 3.16, 5.01, 7.94, 12.6, 20, 31.6, 50.1, 79.4, 126] # E_min (TeV)
bins_hi = [0.0316, 0.0501, 0.0794, 0.126, 0.2, 0.316, 0.501, 0.794, 1.26, 2, 3.16, 5.01, 7.94, 12.6, 20, 31.6, 50.1, 79.4, 126, 200] # E_max (TeV)
data = [2.4e-11, 7.65e-12, 2.16e-12, 1.04e-12, 4.63e-13, 2.98e-13, 1.86e-13, 1.26e-13, 9.76e-14, 7.83e-14, 6.39e-14,
        5.72e-14, 5.75e-14, 7.27e-14, 9.07e-14, 1.14e-13, 1.58e-13, 2.39e-13, 3.64e-13, 5.67e-13] # S (erg / cm^2 / s)
data_GeV = [i * 624.15064799632 for i in data]
mid_point = [np.exp((np.log(i)+np.log(j))/2) * 1000 for i, j in zip(bins_lo, bins_hi)]
plt.loglog(mid_point, data_GeV, color='k', ls='-.', lw='2', alpha=0.5, label = 'CTA South sensivity 50h')

plt.gca().set_ylim(bottom= 1e-12)
plt.gca().set_xlim(left= 1e-1)

plt.loglog(e1, (e1**2*s1).to('GeV cm-2 s-1'), '-', label='Total volume of A3560')
#plt.loglog(e2, (e2**2*s2).to('GeV cm-2 s-1'), '-', label='<R500, spherical integration')
#plt.loglog(e3, (e3**2*s3).to('GeV cm-2 s-1'), '-', label='<R500, cylindrical integration')
plt.xlabel('Energy (GeV)')
plt.ylabel('Flux ($GeV/cm^2/s$)')
plt.legend()

In [ ]:
clust_three = model.Cluster(name='A3562', 
                      redshift=0.049, M500=3.51e14*u.Msun, 
                      cosmology=cosmo, silent=False, 
                      output_dir='/Users/vdk/Desktop/test')

In [ ]:
#----- Some parameters that can be set at when defining the cluster
clust_three.coord = SkyCoord("203.098d  -33.139d", frame="icrs", unit="deg")

#----- Define the relevant ICM model (available models are defined in model_modpar.py)

# Density profile of the target thermal gas
clust_three.density_gas_model = {'name':'beta', 'n_0':0.46e-2*u.cm**-3, 'beta':0.58, 'r_c':223*u.kpc}

# Pressure profile of the target thermal gas
#clust_three.pressure_gas_model = {'name':'GNFW', 'P_0':2.2e-2*u.keV/u.cm**3, 'c500':3.2, 'a':1.5, 'b':3.1, 'c':0.0}

#Gong 20
clust_three.pressure_gas_model = {'name':'GNFW', 'P_0':2.99e-2*u.keV/u.cm**3, 'c500':1.16, 'a':2.66, 'b':5.48, 'c':0.3}

# Set the cosmic ray density profile to the same shape as the thermal gas density
clust_three.set_density_crp_isodens_scal_param()

# Cosmic ray proton spectrum
clust_three.spectrum_crp_model = {'name':'PowerLaw', 'Index':2.2}

# Cosmic ray proton normalization
clust_three.X_crp_E = {'X':0.05, 'R_norm':clust.R500} # X is the CRp/thermal energy ratio, normalized within R_norm

#Set magnetic field profile
clust_three.set_magfield_isodens_scal_param(5*u.uG, scal=0.5)

In [ ]:
energy = np.logspace(-1,5,100)*u.GeV
e1, s1 = clust_three.get_gamma_spectrum(energy,Rmax=clust.R_truncation, type_integral='spherical')
e2, s2 = clust_three.get_gamma_spectrum(energy,Rmax=clust.R500, type_integral='spherical')
e3, s3 = clust_three.get_gamma_spectrum(energy,Rmax=clust.R500, type_integral='cylindrical')

plt.figure(0,figsize=(10,8))

bins_lo = [0.02, 0.0316, 0.0501, 0.0794, 0.126, 0.2, 0.316, 0.501, 0.794, 1.26, 2, 3.16, 5.01, 7.94, 12.6, 20, 31.6, 50.1, 79.4, 126] # E_min (TeV)
bins_hi = [0.0316, 0.0501, 0.0794, 0.126, 0.2, 0.316, 0.501, 0.794, 1.26, 2, 3.16, 5.01, 7.94, 12.6, 20, 31.6, 50.1, 79.4, 126, 200] # E_max (TeV)
data = [2.4e-11, 7.65e-12, 2.16e-12, 1.04e-12, 4.63e-13, 2.98e-13, 1.86e-13, 1.26e-13, 9.76e-14, 7.83e-14, 6.39e-14,
        5.72e-14, 5.75e-14, 7.27e-14, 9.07e-14, 1.14e-13, 1.58e-13, 2.39e-13, 3.64e-13, 5.67e-13] # S (erg / cm^2 / s)
data_GeV = [i * 624.15064799632 for i in data]
mid_point = [np.exp((np.log(i)+np.log(j))/2) * 1000 for i, j in zip(bins_lo, bins_hi)]
plt.loglog(mid_point, data_GeV, color='k', ls='-.', lw='2', alpha=0.5, label = 'CTA South sensivity 50h')

plt.gca().set_ylim(bottom= 1e-12)
plt.gca().set_xlim(left= 1e-1)

plt.loglog(e1, (e1**2*s1).to('GeV cm-2 s-1'), '-', label='Total volume of A3562')
#plt.loglog(e2, (e2**2*s2).to('GeV cm-2 s-1'), '-', label='<R500, spherical integration')
#plt.loglog(e3, (e3**2*s3).to('GeV cm-2 s-1'), '-', label='<R500, cylindrical integration')
plt.xlabel('Energy (GeV)')
plt.ylabel('Flux ($GeV/cm^2/s$)')
plt.legend()

In [ ]:
energy = np.logspace(-1,5,100)*u.GeV
e1, s1 = clust.get_gamma_spectrum(energy,Rmax=clust.R_truncation, type_integral='spherical')
e2, s2 = clust_two.get_gamma_spectrum(energy,Rmax=clust.R_truncation, type_integral='spherical')
e3, s3 = clust_three.get_gamma_spectrum(energy,Rmax=clust.R_truncation, type_integral='spherical')

plt.figure(0,figsize=(10,8))

bins_lo = [0.02, 0.0316, 0.0501, 0.0794, 0.126, 0.2, 0.316, 0.501, 0.794, 1.26, 2, 3.16, 5.01, 7.94, 12.6, 20, 31.6, 50.1, 79.4, 126] # E_min (TeV)
bins_hi = [0.0316, 0.0501, 0.0794, 0.126, 0.2, 0.316, 0.501, 0.794, 1.26, 2, 3.16, 5.01, 7.94, 12.6, 20, 31.6, 50.1, 79.4, 126, 200] # E_max (TeV)
data = [2.4e-11, 7.65e-12, 2.16e-12, 1.04e-12, 4.63e-13, 2.98e-13, 1.86e-13, 1.26e-13, 9.76e-14, 7.83e-14, 6.39e-14,
        5.72e-14, 5.75e-14, 7.27e-14, 9.07e-14, 1.14e-13, 1.58e-13, 2.39e-13, 3.64e-13, 5.67e-13] # S (erg / cm^2 / s)
data_GeV = [i * 624.15064799632 for i in data]
mid_point = [np.exp((np.log(i)+np.log(j))/2) * 1000 for i, j in zip(bins_lo, bins_hi)]
plt.loglog(mid_point, data_GeV, color='k', ls='-.', lw='2', alpha=0.5, label = 'CTA South sensivity 50h')

plt.gca().set_ylim(bottom= 1e-12)
plt.gca().set_xlim(left= 1e-1)

plt.loglog(e1, (e1**2*s1).to('GeV cm-2 s-1'), '-', label='Total volume of A3558')
plt.loglog(e2, (e2**2*s2).to('GeV cm-2 s-1'), '-', label='Total volume of A3560')
plt.loglog(e3, (e3**2*s3).to('GeV cm-2 s-1'), '-', label='Total volume of A3562')
plt.xlabel('Energy (GeV)')
plt.ylabel('Flux ($GeV/cm^2/s$)')
plt.legend()